### Clinical BCI Challenge-WCCI2020
- [website link](https://sites.google.com/view/bci-comp-wcci/?fbclid=IwAR37WLQ_xNd5qsZvktZCT8XJerHhmVb_bU5HDu69CnO85DE3iF0fs57vQ6M)


 - [Dataset Link](https://github.com/5anirban9/Clinical-Brain-Computer-Interfaces-Challenge-WCCI-2020-Glasgow)
 
 
 - [Braindecode Tutorial](https://braindecode.org/auto_examples/plot_bcic_iv_2a_moabb_trial.html)

In [1]:
import braindecode

In [2]:
import mne
from scipy.io import loadmat
import scipy
import sklearn
import numpy as np
import pandas as pd
import glob
from mne.decoding import CSP
import os

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda

In [4]:
import warnings
warnings.filterwarnings('ignore') # to ignore warnings

In [5]:
verbose = False                    # global variable to suppress output display of MNE functions
mne.set_log_level(verbose=verbose) # to suppress large info outputs

In [6]:
n_jobs = None  # for multicore parallel processing, set it to 1 if cause memory issues, for full utilization set to -1

## Data Loading and Conversion to MNE Datatypes
[Mike Cohen Tutorials link for EEG Preprocessing](https://www.youtube.com/watch?v=uWB5tjhataY&list=PLn0OLiymPak2gDD-VDA90w9_iGDgOOb2o)

In [7]:
current_folder = globals()['_dh'][0]  # a hack to get path of current folder in which jupyter file is located
data_path = os.path.join(current_folder, r"C:\Users\MILAKUL\Documents\Thesis\clinicalBCI\matlab")

In [8]:
training_files   = glob.glob(data_path + '/*T.mat')
len(training_files)     # if  return zero,then no file is loaded

10

In [9]:
# we have modified the labels values from [1, 2] to [0, 1] as pytorch 
# expects labels/classes to be in [0, n_classes-1] format
def get_mne_epochs(filepath, verbose=verbose, t_start=2, fs=512, mode='train'):
    '''
    This function reads the EEG data from .mat file and convert it to MNE-Python Compatible epochs
    data structure. It takes data from [0, 8] sec range and return it by setting t = 0 at cue onset
    i.e. 3 seconds and dropping first two seconds so the output data is in [-1.0, 5.0] sec range. The
    Details can be found in the preprocessing section of the attached document
    '''
    mat_data = loadmat(filepath) # read .mat file
    eeg_data= mat_data['RawEEGData']
    idx_start = fs*t_start      
    eeg_data = eeg_data[:, :, idx_start:]
    event_id = {'left-hand': 0, 'right-hand': 1} # pytorch expects labels in [0, n_classes-1]
    channel_names = ['F3', 'FC3', 'C3', 'CP3', 'P3', 'FCz', 'CPz', 'F4', 'FC4', 'C4', 'CP4', 'P4']
    info = mne.create_info(ch_names=channel_names, sfreq=fs, ch_types='eeg')
    epochs = mne.EpochsArray(eeg_data, info, verbose=verbose, tmin=t_start-3.0)
    epochs.set_montage('standard_1020')
    epochs.filter(1., None) 
    epochs.apply_baseline(baseline=(-.250, 0)) # linear baseline correction
    
    if mode == 'train': # this in only applicable for training data
        epochs.event_id = event_id 
        epochs.events[:,2] = mat_data['Labels'].ravel() - 1    
    return epochs 

def get_labels(filepath):
    mat_data = loadmat(filepath) # read .mat file
    return mat_data['Labels'].ravel() - 1

In [10]:
epochs, labels = get_mne_epochs(training_files[0], verbose=verbose), get_labels(training_files[0])
data = epochs.get_data()
print('Shape of EEG Data: ', data.shape, '\t Shape of Labels: ', labels.shape) 

Shape of EEG Data:  (80, 12, 3072) 	 Shape of Labels:  (80,)


### Training Data

In [11]:
# loading original data
epochs_list_train = []
for i in training_files:
    epochs_list_train.append(get_mne_epochs(i, verbose=verbose))

## Deep Learning with Braindecode 

### It's Training Time with [0.5, 4.5] sec and 2sec window with 125ms stride

In [12]:
from braindecode.datautil import create_from_mne_epochs

window_size = 1024   # 2 sec windows
window_stride = 64   # 125 ms stride

windows_datasets_list = []

for epoch in epochs_list_train:
    # Create windows per subject
    windows_dataset = create_from_mne_epochs(
        [epoch.crop(tmin=0.5, tmax=4.5, include_tmax=False)],
        window_size_samples=window_size,
        window_stride_samples=window_stride,
        drop_last_window=False
    )
    # Add labels as a separate attribute
    windows_dataset.update_description = pd.DataFrame(
        data=np.concatenate([d.y for d in windows_dataset.datasets]),
        columns=['labels']
    )
    windows_datasets_list.append(windows_dataset)

print("Datasets per subject:", len(windows_datasets_list))
print("Total windows for first subject:", len(windows_datasets_list[0]))


Datasets per subject: 10
Total windows for first subject: 1360


In [13]:
from braindecode.preprocessing import exponential_moving_standardize

low_cut_hz = 8.   # low cut frequency for filtering
high_cut_hz = 32. # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

def custom_exp_moving_std_fn(epochs, factor_new=factor_new, init_block_size=init_block_size):
    """Apply exponential moving standardization to MNE epochs inplace."""
    data = epochs.get_data()
    for i in range(len(data)):
        data[i] = exponential_moving_standardize(
            data[i], factor_new=factor_new, init_block_size=init_block_size
        )
    epochs._data = data
    return epochs

# Apply preprocessing to each dataset
for windows_dataset in windows_datasets_list:
    # Extract the underlying MNE Epochs object
    epochs = windows_dataset.datasets[0].windows
    epochs.load_data()  # Ensure data is loaded into memory

    # 1) Keep only EEG channels
    epochs.pick_types(eeg=True)

    # 2) Bandpass filter
    epochs.filter(l_freq=low_cut_hz, h_freq=high_cut_hz)

    # 3) Exponential moving standardization
    custom_exp_moving_std_fn(epochs, factor_new=factor_new, init_block_size=init_block_size)


In [16]:
batch_size = 32 #64
n_epochs = 25 #25 #20 #25 use few epochs for quick verification

In [17]:
# Creating a model
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet, EEGNetv4

cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True
seed = 20200220  # random seed to make results reproducible
# Set random seed to be able to reproduce results
set_random_seeds(seed=seed, cuda=cuda)

n_classes=2
# Extract number of chans and time steps from dataset
n_chans = windows_datasets_list[0][0][0].shape[0]
input_window_samples = windows_datasets_list[0][0][0].shape[1]

model = EEGNetv4(
    n_chans,
    n_classes,
    n_times = window_size, #input_window_samples,
    final_conv_length='auto',
)

# Send model to GPU
if cuda:
    model.cuda()

In [18]:
# Training time
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier

lr = 1 * 0.02 #0.01 
weight_decay = 0.5 * 0.001

clfs_list = []
for i in range(len(epochs_list_train)):
    clfs_list.append(
        EEGClassifier(
                    model,
                    criterion = torch.nn.CrossEntropyLoss(),
                    optimizer=torch.optim.AdamW,
                    #train_split=predefined_split(train_set),  # using valid_set for validation
                    optimizer__lr=lr,
                    optimizer__weight_decay=weight_decay,
                    batch_size=batch_size,
                    callbacks=[
                        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
                    ],
                    device=device,
                    )
                )

In [ ]:
all_subject_acc = []
all_subject_kappa = []

def training_function(subject_index=0):
    print('\n', '#' * 25, 'Training for Subject:', subject_index + 1, '#' * 25, '\n')
    
    dataset = windows_datasets_list[subject_index]
    
    y = np.concatenate([d.y for d in dataset.datasets])
    
    clfs_list[subject_index].fit(dataset, y=y, epochs=n_epochs)
    
    history = clfs_list[subject_index].history
    valid_accs = [entry['valid_accuracy'] for entry in history if 'valid_accuracy' in entry]
    
    if valid_accs:
        best_val_acc = max(valid_accs)
        best_val_kappa = (2 * best_val_acc) - 1
        print(f"Best Cross Validation Accuracy: {best_val_acc:.3f}")
        print(f"Best Cross Validation Kappa Score: {best_val_kappa:.3f}")
        
      
        all_subject_acc.append(best_val_acc)
        all_subject_kappa.append(best_val_kappa)
    else:
        print("Validation accuracy not available in history.")


for sub_idx in range(len(windows_datasets_list)):
    training_function(sub_idx)

mean_acc = np.mean(all_subject_acc)
mean_kappa = np.mean(all_subject_kappa)
print("\n" + "="*50)
print(f"Mean Within-Subject Accuracy: {mean_acc:.3f}")
print(f"Mean Within-Subject Kappa:    {mean_kappa:.3f}")
print("="*50)




 ######################### Training for Subject: 1 ######################### 

Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  -----------  ----------------  ------------  ------  ------
      1            0.5064        0.6766       0.4375            0.4375        6.6486  0.0200  1.9657
      2            0.5055        0.2789       0.4375            0.4375        7.3016  0.0199  1.6942
      3            0.5000        0.2826       0.4375            0.4375       11.3223  0.0197  1.7398
      4            0.7077        0.2146       0.5404            0.5404        2.6112  0.0192  1.7120
      5            0.9642        0.1990       0.6691            0.6691        3.6753  0.0187  1.6859
      6            0.5506        0.1096       0.4375            0.4375        7.9458  0.0179  1.6684
      7            0.6342        0